In [1]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver('weigelt_solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [2]:
def get_lms(factory, minv=0.0, maxv=solver.infinity()):
    l = solver.IntVar(minv, maxv, 'units of large size for factory %s' % factory)
    m = solver.IntVar(minv, maxv, 'units of medium size for factory %s' % factory)
    s = solver.IntVar(minv, maxv, 'units of small size for factory %s' % factory)
    return l, m, s

In [3]:
l1,m1,s1 = get_lms("1")
l2,m2,s2 = get_lms("2")
l3,m3,s3 = get_lms("3")

In [4]:
# factory 1 can produce up to 750
# l1 + m1 + s1 <= 750
constraint1 = solver.Constraint(0.0, 750)
constraint1.SetCoefficient(l1, 1)
constraint1.SetCoefficient(m1, 1)
constraint1.SetCoefficient(s1, 1)

In [5]:
# factory 2 can produce up to 900
# l2 + m2 + s2 <= 900
solver.Add(l2+m2+s2 <= 900);

In [6]:
# factory 3 can produce up to 450
# l3 + m3 + s3 <= 450
solver.Add(l3 + m3 + s3 <= 450);

In [7]:
# factory 1 has capacity for 13000
solver.Add(20*l1 + 15*m1 + 12*s1 <= 13000);

In [8]:
# factory 2 has capacity for 13000
solver.Add(20*l2 + 15*m2 + 12*s2 <= 12000);

In [9]:
# factory 3 has capacity for 5000
solver.Add(20*l3 + 15*m3 + 12*s3 <= 5000);

In [10]:
solver.Add(l1+l2+l3<=900);
solver.Add(m1+m2+m3<=1200);
solver.Add(s1+s2+s3<=750);

In [11]:
solver.Add((l1+m1+s1)/750 == (l2+m2+s2)/900);
solver.Add((l1+m1+s1)/750 == (l3+m3+s3)/450);

In [12]:
solver.Maximize(420*(l1+l2+l3) + 360*(m1+m2+m3) + 300*(s1+s2+s3))

In [13]:
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

Number of variables = 9
Number of constraints = 11


In [14]:
result_status = solver.Solve()

In [15]:
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

In [21]:
print('Optimal objective value = %d' % solver.Objective().Value())

Optimal objective value = 694680


In [26]:
variable_list = [l1,m1,s1,l2,m2,s2,l3,m3,s3]
for variable in variable_list:
    print('%35s = %4d' % (variable.name(), variable.solution_value()))

  units of large size for factory 1 =  530
 units of medium size for factory 1 =  160
  units of small size for factory 1 =    0
  units of large size for factory 2 =    0
 units of medium size for factory 2 =  688
  units of small size for factory 2 =  140
  units of large size for factory 3 =    0
 units of medium size for factory 3 =   10
  units of small size for factory 3 =  404


In [20]:
# solver.__dir__()